In [22]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sparse
from implicit.evaluation import *

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/Khoande150820/steam_game_recommender/main/steam_users_clean.csv')
df = df.rename(columns={'user': 'userid', 'ID': 'gameid'})
df.head(10)

,userid,game,hours,purchase,play,gameid
0,5250,Alien Swarm,4.9,1,1,alienswarm
1,5250,Cities Skylines,144.0,1,1,citiesskylines
2,5250,Counter-Strike,0.0,1,0,counterstrike
3,5250,Counter-Strike Source,0.0,1,0,counterstrikesource
4,5250,Day of Defeat,0.0,1,0,dayofdefeat
5,5250,Deathmatch Classic,0.0,1,0,deathmatchclassic
6,5250,Deus Ex Human Revolution,62.0,1,1,deusexhumanrevolution
7,5250,Dota 2,0.2,1,1,dota2
8,5250,Half-Life,0.0,1,0,halflife
9,5250,Half-Life 2,0.0,1,0,halflife2


In [25]:
df['userid'] = df['userid'].astype('category')
df['gameid'] = df['gameid'].astype('category')
df['userid'] = df['userid'].cat.codes
df['gameid'] = df['gameid'].cat.codes
df.head(10)

,userid,game,hours,purchase,play,gameid
0,0,Alien Swarm,4.9,1,1,209
1,0,Cities Skylines,144.0,1,1,840
2,0,Counter-Strike,0.0,1,0,966
3,0,Counter-Strike Source,0.0,1,0,972
4,0,Day of Defeat,0.0,1,0,1115
5,0,Deathmatch Classic,0.0,1,0,1157
6,0,Deus Ex Human Revolution,62.0,1,1,1237
7,0,Dota 2,0.2,1,1,1331
8,0,Half-Life,0.0,1,0,2068
9,0,Half-Life 2,0.0,1,0,2069


In [26]:
sparse_game_user = sparse.csr_matrix((df['hours'].astype(float), (df['gameid'], df['userid'])))
sparse_user_game = sparse.csr_matrix((df['hours'].astype(float), (df['userid'], df['gameid'])))

rate_matrix_train, rate_matrix_test = implicit.evaluation.train_test_split(sparse_user_game, 0.9)

model = implicit.als.AlternatingLeastSquares(factors=32, 
                                             regularization=0.1, 
                                             iterations=500)
alpha_val = 15
data_conf = (rate_matrix_train * alpha_val).astype('double')

model.fit(data_conf)

  0%|          | 0/500 [00:00<?, ?it/s]

In [27]:
game_id = df['gameid'].loc[df.game == 'Grand Theft Auto V'].iloc[0]
n_similar = 30

similar = model.similar_items(game_id, n_similar)

ids, scores = similar

for i in range(n_similar):
    print(df.game.loc[df.gameid == ids[i]].iloc[0], '| score: ' + str(scores[i]))

Grand Theft Auto V | score: 1.0000001
Game Dev Tycoon | score: 0.48674637
Fallout 4 | score: 0.47826537
Arma 3 | score: 0.46471146
Saints Row Gat out of Hell | score: 0.43923676
Batman Arkham Knight | score: 0.43854383
Alien Isolation | score: 0.43137908
Castlevania Lords of Shadow - Ultimate Edition | score: 0.41886964
Take On Helicopters | score: 0.4175262
Towtruck Simulator 2015 | score: 0.4143234
Test Drive Unlimited 2 | score: 0.4129907
F1 2011 | score: 0.4085077
METAL GEAR SOLID V THE PHANTOM PAIN | score: 0.40503442
Autobahn Police Simulator | score: 0.39721018
Chip's Challenge 2 | score: 0.39523223
The Crew | score: 0.3951987
Helicopter Simulator 2014 Search and Rescue | score: 0.39417613
Borderlands The Pre-Sequel | score: 0.38662118
Just Cause 3 | score: 0.38597578
Speedball 2 Tournament | score: 0.38364714
DARK SOULS II Scholar of the First Sin | score: 0.37817597
LEGO Batman The Videogame | score: 0.37369916
Ultimate General Gettysburg | score: 0.36361805
NBA 2K13 | score: 

In [28]:
implicit.evaluation.mean_average_precision_at_k(model, rate_matrix_train, rate_matrix_test)

  0%|          | 0/3004 [00:00<?, ?it/s]

0.11050900407753957

In [29]:
implicit.evaluation.AUC_at_k(model, rate_matrix_train, rate_matrix_test)

  0%|          | 0/3004 [00:00<?, ?it/s]

0.5991836631490969

In [50]:
df.game.loc[df.userid == 5715]

97927            Alice Madness Returns
97928                        Anno 1404
97929                      Antichamber
97930              BattleBlock Theater
97931    Brothers - A Tale of Two Sons
97932                           Dota 2
97933                              FEZ
97934        Fable - The Lost Chapters
97935     Kingdoms of Amalur Reckoning
97936                Lego Harry Potter
97937                         Magicite
97938                          Magicka
97939                           Monaco
97940                 Prison Architect
97941                 RPG Maker VX Ace
97942                    Shadowgrounds
97943           Shadowgrounds Survivor
97944    South Park The Stick of Truth
97945                            Spore
97946        Spore Galactic Adventures
97947             The Binding of Isaac
97948     The Binding of Isaac Rebirth
97949      The Book of Unwritten Tales
97950                         The Cave
97951            The Sims(TM) Medieval
97952     The Witcher Enh

In [42]:
for i in range(len(df.userid.unique())):
  if len(df.game.loc[df.userid == i]) > 10:
    print(i)

0
1
2
5
6
7
8
10
11
12
17
19
22
23
24
25
27
28
31
32
40
43
44
46
47
49
50
51
52
53
54
56
57
59
60
64
68
70
71
72
73
75
76
79
80
85
89
90
91
94
95
97
100
101
102
104
105
106
107
108
110
112
114
116
118
119
121
122
123
124
125
126
127
130
135
137
139
141
143
144
146
147
148
150
151
152
155
158
160
161
162
167
168
171
172
173
178
179
180
181
182
185
186
187
191
192
199
202
204
205
206
208
209
210
211
219
220
221
222
223
224
225
227
230
231
233
234
237
240
241
242
243
244
246
249
254
255
257
264
267
271
274
276
277
278
279
280
285
286
288
289
290
291
293
302
303
306
311
312
313
314
315
316
321
328
330
333
335
336
337
340
341
346
347
348
349
350
351
354
364
366
371
374
375
377
378
381
383
384
387
393
395
397
398
400
405
406
407
408
409
418
420
427
428
431
432
443
446
450
451
454
455
459
461
466
467
470
473
478
480
482
483
486
490
492
494
500
511
517
518
519
521
530
532
535
536
549
553
554
555
557
560
565
584
595
596
599
610
612
618
621
628
633
638
642
647
648
654
656
657
662
665
670
680
685

In [36]:
n_similar = 20
ids, score = model.recommend(0, sparse_user_game[0], n_similar)

for i in range(n_similar):
    print(df.game.loc[df.gameid == ids[i]].iloc[0], '| score: ' + str(scores[i]))

Left 4 Dead 2 | score: 1.0000001
Metro 2033 | score: 0.48674637
Saints Row The Third | score: 0.47826537
Left 4 Dead | score: 0.46471146
PAYDAY The Heist | score: 0.43923676
Empire Total War | score: 0.43854383
Terraria | score: 0.43137908
Grand Theft Auto IV | score: 0.41886964
Dishonored | score: 0.4175262
Mirror's Edge | score: 0.4143234
South Park The Stick of Truth | score: 0.4129907
Assassin's Creed II | score: 0.4085077
Torchlight | score: 0.40503442
The Elder Scrolls V Skyrim | score: 0.39721018
Borderlands 2 | score: 0.39523223
DC Universe Online | score: 0.3951987
XCOM Enemy Unknown | score: 0.39417613
Torchlight II | score: 0.38662118
Dead Island | score: 0.38597578
Universe Sandbox | score: 0.38364714
